**1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data or newtype) for each of them. **

In [ ]:
type Date = Int String Int
type Person = String String Date